<a href="https://colab.research.google.com/github/AldiAlfatih/Program-Sistem-Cerdas-PTC/blob/main/Sistem_Prediksi_Kualitas_Panen_Hidroponik_Selada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn
!pip install tensorflow
!pip show tensorflow
# !pip uninstall scikit-learn
!pip install scikit-learn==1.4.2
!pip install scikeras

Name: tensorflow
Version: 2.17.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 44.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasClassifier
from scipy.stats import randint
from sklearn.metrics import confusion_matrix

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Vinzzztty/playground-data-analyst/main/Machine%20Learning%20Terapan/2.%20Sentiment%20Analysis/dataset/dataset_selada.csv")

In [ ]:
df.head()

In [ ]:
# Memilih fitur
fitur = ['temperature', 'humidity', 'TDS', 'WaterTemp']
X = df[fitur]

In [ ]:
X.head(10)

In [ ]:
# Menentukan jumlah cluster
n_clusters = 5

In [ ]:
# Melakukan clustering dengan K-Means
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['Kategori'] = kmeans.fit_predict(X)

In [ ]:
# Membagi data
X = df[fitur]
y = df['Kategori']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Normalisasi Data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print("TensorFlow version:", tf.__version__)
print("Scikit-learn version:", sklearn.__version__)
print("SciKeras version:", scikeras.__version__)

In [ ]:
def create_lstm_model(units=60, dropout_rate=0.3, optimizer='adam'):
    model = Sequential()
    model.add(Dense(units, activation='relu', input_dim=X_train_scaled.shape[1]))  # Layer Dense pertama
    model.add(Dense(units, activation='relu'))  # Layer Dense kedua
    model.add(Dense(n_clusters, activation='softmax'))  # Layer output
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Buat model LSTM
model_lstm = create_lstm_model(
    units=60,
    dropout_rate=0.3,  # Dropout tidak digunakan di Dense layer
    optimizer='rmsprop'
)

# Latih model LSTM
model_lstm.fit(X_train_scaled, y_train, epochs=10, batch_size=64, verbose=2)

# Evaluasi model LSTM pada data uji
loss, accuracy = model_lstm.evaluate(X_test_scaled, y_test, verbose=0)
print('Loss:', loss)
print('Accuracy:', accuracy)

In [ ]:
# Evaluasi model LSTM pada data uji
loss, accuracy = model_lstm.evaluate(X_test_scaled, y_test, verbose=0)
print('Loss:', loss)
print('Accuracy:', accuracy)

# Menampilkan hasil prediksi pada data latih (LSTM)
y_pred_train_lstm = np.argmax(model_lstm.predict(X_train_scaled), axis=1)
print("Hasil prediksi pada data latih (LSTM):")
print(y_pred_train_lstm)

# Menampilkan hasil prediksi pada data uji (LSTM)
y_pred_lstm = np.argmax(model_lstm.predict(X_test_scaled), axis=1)
print("Hasil prediksi pada data uji (LSTM):")
print(y_pred_lstm)

# Evaluasi model LSTM pada data uji
print("Evaluasi Model LSTM:")
print(classification_report(y_test, y_pred_lstm))

In [ ]:
def create_ann_model(neurons_1=128, neurons_2=64, activation='relu', optimizer='adam'):
    model = Sequential()
    model.add(Dense(neurons_1, activation=activation, input_dim=X_train_scaled.shape[1]))
    model.add(Dense(neurons_2, activation=activation))
    model.add(Dense(n_clusters, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Buat model ANN dengan parameter yang Anda inginkan
model_ann = create_ann_model(
    neurons_1=200,  # Contoh nilai neurons_1
    neurons_2=100,  # Contoh nilai neurons_2
    activation='sigmoid',  # Contoh fungsi aktivasi
    optimizer='rmsprop'  # Contoh optimizer
)

# Latih model ANN
model_ann.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=2)  # Contoh nilai epochs dan batch_size

# Evaluasi model ANN pada data uji
loss, accuracy = model_ann.evaluate(X_test_scaled, y_test, verbose=0)
print('Loss:', loss)
print('Accuracy:', accuracy)

In [ ]:
# Evaluasi model ANN pada data uji
loss, accuracy = model_ann.evaluate(X_test_scaled, y_test, verbose=0)
print('Loss:', loss)
print('Accuracy:', accuracy)

# Menampilkan hasil prediksi pada data latih (ANN)
y_pred_train_ann = np.argmax(model_ann.predict(X_train_scaled), axis=1)
print("Hasil prediksi pada data latih (ANN):")
print(y_pred_train_ann)

# Menampilkan hasil prediksi pada data uji (ANN)
y_pred_ann = np.argmax(model_ann.predict(X_test_scaled), axis=1)
print("Hasil prediksi pada data uji (ANN):")
print(y_pred_ann)

# Evaluasi model ANN pada data uji
print("Evaluasi Model ANN:")
print(classification_report(y_test, y_pred_ann))

In [ ]:
# Confusion matrix untuk model LSTM
cm_lstm = confusion_matrix(y_test, y_pred_lstm)

# Confusion matrix untuk model ANN
cm_ann = confusion_matrix(y_test, y_pred_ann)

# Visualisasi confusion matrix LSTM
plt.figure(figsize=(8, 6))
sns.heatmap(cm_lstm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - LSTM')
plt.xlabel('Prediksi')
plt.ylabel('Aktual')
plt.show()

# Visualisasi confusion matrix ANN
plt.figure(figsize=(8, 6))
sns.heatmap(cm_ann, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - ANN')
plt.xlabel('Prediksi')
plt.ylabel('Aktual')
plt.show()

In [ ]:
import tensorflow as tf
model_ann.save('best_model_ann.h5')
model = tf.keras.models.load_model('best_model_ann.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tfliteModel = converter.convert()

with open('best_model_ann.tflite', 'wb') as f:
    f.write(tfliteModel)